In [ ]:
import pandas as pd
import numpy as np


In [ ]:
#Load the dataset
#file_path = os.path.join(os.getcwd(), "Data", "raw", "student_depression_dataset.csv")
#df = pd.read_csv(file_path)

df=pd.read_csv("./Data/raw/student_depression_dataset.csv")

In [ ]:
df.head()

Check the overall details about the dataset

In [ ]:
# Print basic info about the dataset
print(f"   Information of the raw dataset \n -------------------------------------------\n")
df.info()

Check whether there are missing values in the dataset

In [ ]:
#Check missing values
df.isnull().sum()

Check whether there are duplicates in the dataset

In [ ]:
#Check for duplicate rows
df.duplicated().sum()

Drop Irrelevant Columns

In [ ]:
df.drop(columns=['id'],inplace=True)

Identify Feature Types

In [ ]:
#Target Variable
target='Depression'

#Features(excluding target)
features=df.drop(columns=[target])

#Categorical Features
categorical_cols=features.select_dtypes(include='object').columns.tolist()

#Numerical Features
numerical_cols=features.select_dtypes(exclude='object').columns.tolist()

print("Categorical:",categorical_cols)
print("Numerical:",numerical_cols)

Encode Categorical Variables
SVM only works with numerical data. So need to encode categorical data

01.Label Encode Binary Categorical Columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


#Apply label encoding to binary categorical columns
binary_cols=['Gender','Academic Pressure','Work Pressure',
               'Study Satisfaction','Job Satisfaction',
               'Dietary Habits','Have you ever had suicidal thoughts ?',
               'Financial Stress','Family History of Mental Illness']

for col in binary_cols:
    df[col] =le.fit_transform(df[col])

02.One-Hot encode nominal categorical Columns

In [ ]:
df=pd.get_dummies(df,columns=['City','Profession','Degree'],drop_first=True)
